In [619]:
import cv2 as cv
import sys
import numpy as np
import math as mt
from matplotlib import pyplot as plt
import glob

np.random.seed(0)

In [620]:
def SaveImage(img,name,dst=''):
    cv.imwrite(dst+name,img)

In [621]:
def ProcessBileteral(image,name):
    bileteral=cv.bilateralFilter(image, 15 ,  75 , 75)
    SaveImage(bileteral,name,'method/bileteral/denoising/')
    return bileteral

def ProcessErode(image,name,kernel):
    erode=cv.erode(image,kernel)
    SaveImage(image,name,'method/erosion/denoising/')
    return erode

In [622]:
def CalculateMask(image,name,dst):

    hsv_image=cv.cvtColor(image,cv.COLOR_BGR2HSV)

    healty_part=cv.inRange(hsv_image,(36,10,25),(86,255,255))
    background=cv.inRange(hsv_image,(90,5,25),(240,255,255))+cv.inRange(hsv_image,(0,5,31),(2,255,255))
    shadows=cv.inRange(hsv_image,(0,0,0),(240,255,25))
    
    leafs_mark_vis=np.zeros_like(image,np.uint8)
    leafs_mark_vis[healty_part>0]=(255,0,255)

    background_mark_vis=np.zeros_like(image,np.uint8)
    background_mark_vis[background>0]=(255,255,255)

    shadows_mark_vis=np.zeros_like(image,np.uint8)
    shadows_mark_vis[shadows>0]=(0,0,255)

    markers_visualization=leafs_mark_vis+background_mark_vis+shadows_mark_vis

    SaveImage(np.concatenate([markers_visualization,image],axis=1),name,dst+'markers/')
    

    healty_mask=np.random.choice(a=[True, False], size=(image.shape[0],image.shape[1]), p=[1, 0])  
    background_shadows_mask=np.random.choice(a=[True, True], size=(image.shape[0],image.shape[1]), p=[1, 0]) 
    markers=np.zeros((image.shape[0],image.shape[1]),dtype='int32')

    markers[np.all([healty_part>0,healty_mask==True],axis=0)]=255
    markers[ np.all( [np.any([background>0,shadows>0],axis=0),background_shadows_mask==True],  axis=0) ]=1
    
    labels =cv.watershed(hsv_image,markers)
    print( np.unique(labels))
    colors=[(125,0,255),(0,0,0),(125,255,0),(0,255,255)]
   
    mask=np.zeros_like(image,np.uint8)
    i=0
    for label in np.unique(labels): 
        mask[labels == label] = colors[i]
        i+=1
    mask_leafs_only= np.copy(mask)

    ill_part=labels-healty_part
    mask[ill_part>1]=colors[3]
    conc=np.concatenate([image,markers_visualization,mask_leafs_only,mask],axis=1)
    SaveImage(conc,name,dst+'recognizing/')
    cv.imshow('Display window',conc)
    cv.waitKey(0)





In [623]:
def ProcessImagesBileteral(images_paths):
    for image_path in images_paths:
        image=cv.imread(image_path)
        name=image_path[image_path.rfind('\\')+1:]
        bileteral=ProcessBileteral(image,name)
        CalculateMask(bileteral,name,'method/bileteral/')

def ProcessImagesErode(images_paths):
    for image_path in images_paths:
        image=cv.imread(image_path)
        name=image_path[image_path.rfind('\\')+1:]
        kernel=cv.getStructuringElement(cv.MORPH_ELLIPSE,(7,7))
        erode=ProcessErode(image,name,kernel)
        CalculateMask(erode,name,'method/erosion/')

In [624]:
images_paths=[path for path in glob.glob('test/*.jpg')]
ProcessImagesBileteral(images_paths)
cv.destroyAllWindows()

[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
